
<h1><left>EN-BOT: Predicting whether a Block goes En-Bloc!</left></h1>
<h1><left>Data Collection</left></h1>


<h4><left>by Samuel He</left></h4>


### Current Question:
What are the reasons why a block is chosen to be demolished.
Can we predict if a block will be demolished. 

### Pivot!:
Recommender System for HDB Flats!
Yes! We actually have this data!

### HTML SCRIPT for Links
Please write it neatly like this.
"Today, we're going to be using the [wine quality](http://www3.dsi.uminho.pt/pcortez/wine/) dataset by Cortez, Cerdeira, Almeida, Matos and Reis."

### Links to Our APIs
Help from:
- https://github.com/admond1994/onemap-API/blob/master/get_token_API.py
- WHAT THE HECK THIS GUY MADE A WRAPPER: https://github.com/methylDragon/onemapsg-api-client

### Data sources:
I'm currently working with this amazingly detailed dataset containing all the standing blocks: https://data.gov.sg/dataset/hdb-property-information (Thanks for this!)

I'm hoping to get hold of similar details for SERS blocks. But for now, The HDB website has released general details of SERS clusters on their site. But there's no information there on the make-up of each block, how many floors,  year built, etc. 
https://www.hdb.gov.sg/cs/infoweb/residential/living-in-an-hdb-flat/sers-and-upgrading-programmes/completed-sers-projects

### More Data!
Maybe Height building limiits? If a block is already at max-height. it will not be able to build higher.

### FINAL OUTPUT SHOULD BE A One PAGE WEBSITE


In [1]:
import requests
import time
import pandas as pd
import numpy as np
from random import randint
from onemapsg import OneMapClient

# UNCOMMENT TO INSTALL THE ONEMAPSG API WRAPPER
# !pip3 install --upgrade --target=/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages onemapsg

## 1. First Looks
- We take any early peek into our two datasets


In [2]:
#READ IN STANDING BLOCKS
pd.set_option('display.max_columns', None)
stand_blocks = pd.read_csv("../data/hdb_standing.csv")
stand_blocks

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,KWN,142,0,1,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,BD,206,0,0,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,CT,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,BD,102,0,0,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,PRC,55,0,0,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12195,998A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,HG,185,0,102,33,50,0,0,0,0,0,0,0,0
12196,998B,BUANGKOK CRES,17,2018,Y,N,N,N,N,N,HG,174,0,96,31,47,0,0,0,0,0,0,0,0
12197,999,BUANGKOK CRES,2,2018,N,N,N,Y,N,Y,HG,0,0,0,0,0,0,0,0,0,0,0,0,0
12198,999A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,HG,306,0,0,0,0,0,0,0,0,136,170,0,0


In [3]:
#LOWERCAS-ING THE ENTIRE DATAFRAME
stand_blocks = stand_blocks.apply(lambda x: x.str.lower() if x.dtype == "object" else x)  
stand_blocks

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,beach rd,16,1970,y,y,n,n,n,n,kwn,142,0,1,138,1,2,0,0,0,0,0,0,0
1,1,bedok sth ave 1,14,1975,y,n,n,y,n,n,bd,206,0,0,204,0,2,0,0,0,0,0,0,0
2,1,cantonment rd,2,2010,n,y,n,n,n,n,ct,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,chai chee rd,15,1982,y,n,n,n,n,n,bd,102,0,0,0,10,92,0,0,0,0,0,0,0
4,1,changi village rd,4,1975,y,y,n,n,n,n,prc,55,0,0,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12195,998a,buangkok cres,18,2018,y,n,n,n,n,n,hg,185,0,102,33,50,0,0,0,0,0,0,0,0
12196,998b,buangkok cres,17,2018,y,n,n,n,n,n,hg,174,0,96,31,47,0,0,0,0,0,0,0,0
12197,999,buangkok cres,2,2018,n,n,n,y,n,y,hg,0,0,0,0,0,0,0,0,0,0,0,0,0
12198,999a,buangkok cres,18,2018,y,n,n,n,n,n,hg,306,0,0,0,0,0,0,0,0,136,170,0,0


In [4]:
#AS WE CAN SEE, DATA IS VERY COMPLETE HERE, WE WILL EVENTUALLY HAVE TO REMOVE SOME COLUMNS THOUGH 
stand_blocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12200 entries, 0 to 12199
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   blk_no                 12200 non-null  object
 1   street                 12200 non-null  object
 2   max_floor_lvl          12200 non-null  int64 
 3   year_completed         12200 non-null  int64 
 4   residential            12200 non-null  object
 5   commercial             12200 non-null  object
 6   market_hawker          12200 non-null  object
 7   miscellaneous          12200 non-null  object
 8   multistorey_carpark    12200 non-null  object
 9   precinct_pavilion      12200 non-null  object
 10  bldg_contract_town     12200 non-null  object
 11  total_dwelling_units   12200 non-null  int64 
 12  1room_sold             12200 non-null  int64 
 13  2room_sold             12200 non-null  int64 
 14  3room_sold             12200 non-null  int64 
 15  4room_sold         

In [5]:
#READ IN DEMOLISHED BLOCKS
demo_all = pd.read_csv("../data/hdb_demolish.csv")

In [6]:
#NaNs ABOUND! 
#HDB DID NOT MAKE A LOT OF DATA ABOUT ITS 70 CLUSTERS OF DEMOLISHED BLOCKS AVAILABLE. 
demo_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Site No                     81 non-null     object 
 1   SERS Announced              84 non-null     object 
 2   town                        84 non-null     object 
 3   street                      84 non-null     object 
 4   cluster                     84 non-null     object 
 5   year_completed              39 non-null     float64
 6   ave_floors                  30 non-null     float64
 7   1room_sold                  39 non-null     float64
 8   2room_sold                  39 non-null     float64
 9   3room_sold                  39 non-null     float64
 10  4room_sold                  39 non-null     float64
 11  5room_sold                  39 non-null     float64
 12  exec_sold                   39 non-null     float64
 13  total_dwelling_units        41 non-nu

## 2. Working on the "demolished blocks" dataset
As the "demolished blocks" dataset has way less features and is dirtier. We should aim to clean it first and create corresponding features that match that of the *stand_blocks* dataset.

### 2.1. Formatting
We'll fix the format of some of our columns and also drop the columns we will not be using. We'll end this segment by making a decision about our columns with too many nulls. 

In [7]:
#MAKING EVERYTHING LOWERCASE
#CLEANING UP THE DATETIME FORMAT
#EXTRACTING THE YEAR AND MAKING THAT A COLUMN
demo_all = demo_all.apply(lambda x: x.str.lower() if x.dtype == "object" else x)  
demo_all["SERS Announced"] = pd.to_datetime(demo_all["SERS Announced"])
demo_all["sers_year"] = [x.year for x in pd.DatetimeIndex(demo_all["SERS Announced"])]
demo_all

,Site No,SERS Announced,town,street,cluster,year_completed,ave_floors,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,Others,Replacement Street Name,Replacement Site Block No.,Replacement Storey height,replacement No of units,sers_year
0,1,1995-08-22,bukit merah - tiong bahru,boon tiong road,"[2, 4, 6, 8, 10, 12, 13 ]",1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,384.0,NaN,boon tiong road,"2a, 2c",NaN,"706 4-rm, 696 5-rm",1995
1,NaN,1995-08-22,bukit merah - tiong bahru,boon tiong road,"[1, 3, 5, 7, 9, 11, 14, 15 , 16]",1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boon tiong road,"4a, 8a, 8b",NaN,NaN,1995
2,2,1995-09-01,toa payoh,lorong 2 toa payoh,"[145, 147, 151 ,152]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,lorong 2 toa payoh,"141, 153, 153a, 154 to 156",NaN,?,1995
3,3,1995-11-08,bukit merah,depot road,"[103, 110 ,111,105, 107 , 112 ]",1975.0,NaN,0.0,0.0,822.0,176.0,5.0,0.0,1003.0,NaN,depot road,"103a, 103b, 104a, 104b, 106a , 106b",NaN,?,1995
4,4,1996-02-08,bukit merah,redhill road,"[67 , 71,72, 77 ]",NaN,NaN,228.0,0.0,384.0,0.0,0.0,0.0,612.0,#error!,redhill close,68 to 70,NaN,247 + 15 shops,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,76,2011-12-03,bukit merah,redhill close,"[1,2, 3, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,...",1955.0,7.00,0.0,0.0,875.0,3.0,0.0,0.0,878.0,NaN,henderson road,"blocks 95a to 95c, 96a , 96b\n(city vue @hende...",NaN,(~1200 units),2011
80,77,2011-12-29,jurong west - boon lay gardens,boon lay drive,"[167,168,169,170,171, 172]",1973.0,11.00,0.0,0.0,640.0,0.0,10.0,0.0,650.0,NaN,jurong west central,"697a to 6097c, 698a to 698c\n(jurong west blos...",16,(~710 units),2011
81,78,2012-06-25,woodlands,woodlands centre road,"[""1a"" , ""2a""]",1983.0,4.00,0.0,0.0,143.0,0.0,4.0,0.0,147.0,#error!,"woodl,s drive 70","blocks 717a , 717b\n(admiralty vista)",13/16,(~190 units),2012
82,79,2014-06-27,queenstown,tanglin halt road,"[24,25,26,27,28,29,30,31,32, 33,34,35,36,37, 3...",1962.0,9.50,0.0,837.0,2539.0,104.0,0.0,0.0,3480.0,NaN,margaret drive (site a)\nmargaret drive (site ...,NaN,NaN,(~3707),2014


In [8]:
#SHORT-FORMING street COLUMN TO MATCH THAT OF stand_blocks
demo_all["street"] = demo_all["street"].replace({"road": "rd", "drive":"dr", "close":"cl", "crescent":"cres", "avenue":"ave"}, regex=True)

In [9]:
#WE'LL WORK WITHOUT THE LAST 5 COLUMNS
#THEY CONTAIN INFO ABOUT REPLACEMENT BLOCKS
#(WHICH IS INTERESTING BUT CURRENTLY NOT AS USEFUL)

# !!!!
#WE'VE ADDED ROWS SO THIS DOES NOT WORK ANYMORE. PLS JUST DROP ROWS!

demo_blocks = demo_all.drop(["Others",
                             "Replacement Street Name",
                             "Replacement Site Block No.",
                             "Replacement Storey height",
                             "replacement No of units"], 
                            axis="columns")
demo_blocks

,Site No,SERS Announced,town,street,cluster,year_completed,ave_floors,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,sers_year
0,1,1995-08-22,bukit merah - tiong bahru,boon tiong rd,"[2, 4, 6, 8, 10, 12, 13 ]",1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,384.0,1995
1,NaN,1995-08-22,bukit merah - tiong bahru,boon tiong rd,"[1, 3, 5, 7, 9, 11, 14, 15 , 16]",1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1995
2,2,1995-09-01,toa payoh,lorong 2 toa payoh,"[145, 147, 151 ,152]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,1995
3,3,1995-11-08,bukit merah,depot rd,"[103, 110 ,111,105, 107 , 112 ]",1975.0,NaN,0.0,0.0,822.0,176.0,5.0,0.0,1003.0,1995
4,4,1996-02-08,bukit merah,redhill rd,"[67 , 71,72, 77 ]",NaN,NaN,228.0,0.0,384.0,0.0,0.0,0.0,612.0,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,76,2011-12-03,bukit merah,redhill cl,"[1,2, 3, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,...",1955.0,7.00,0.0,0.0,875.0,3.0,0.0,0.0,878.0,2011
80,77,2011-12-29,jurong west - boon lay gardens,boon lay dr,"[167,168,169,170,171, 172]",1973.0,11.00,0.0,0.0,640.0,0.0,10.0,0.0,650.0,2011
81,78,2012-06-25,woodlands,woodlands centre rd,"[""1a"" , ""2a""]",1983.0,4.00,0.0,0.0,143.0,0.0,4.0,0.0,147.0,2012
82,79,2014-06-27,queenstown,tanglin halt rd,"[24,25,26,27,28,29,30,31,32, 33,34,35,36,37, 3...",1962.0,9.50,0.0,837.0,2539.0,104.0,0.0,0.0,3480.0,2014


In [10]:
#CHECKING HOW NaN-INFECTED OUR COLUMNS ARE
demo_blocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Site No               81 non-null     object        
 1   SERS Announced        84 non-null     datetime64[ns]
 2   town                  84 non-null     object        
 3   street                84 non-null     object        
 4   cluster               84 non-null     object        
 5   year_completed        39 non-null     float64       
 6   ave_floors            30 non-null     float64       
 7   1room_sold            39 non-null     float64       
 8   2room_sold            39 non-null     float64       
 9   3room_sold            39 non-null     float64       
 10  4room_sold            39 non-null     float64       
 11  5room_sold            39 non-null     float64       
 12  exec_sold             39 non-null     float64       
 13  total_dwelling_units  

In [11]:
#DATA CLEANING DECISION: CLEAR OUT THE ROWS WITH NULLS
#AS AN UNFORTUNATE RESULT OF SUBJECT MATTER, IT LOOKS LIKE WE ONLY HAVE THIRTY TRULY CLEAN CLUSTERS OF DEMOLISHED BLOCKS
#CAN WE WORK FROM THERE?
demo_blocks_complete= demo_blocks.dropna()
demo_blocks_complete.shape

(30, 15)

In [12]:
#CHECKING FOR NULL VALUES
demo_blocks_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 7 to 83
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Site No               30 non-null     object        
 1   SERS Announced        30 non-null     datetime64[ns]
 2   town                  30 non-null     object        
 3   street                30 non-null     object        
 4   cluster               30 non-null     object        
 5   year_completed        30 non-null     float64       
 6   ave_floors            30 non-null     float64       
 7   1room_sold            30 non-null     float64       
 8   2room_sold            30 non-null     float64       
 9   3room_sold            30 non-null     float64       
 10  4room_sold            30 non-null     float64       
 11  5room_sold            30 non-null     float64       
 12  exec_sold             30 non-null     float64       
 13  total_dwelling_units  

### 2.2 Prepping  "demolished blocks" dataset to be matched with "standing blocks" dataset
We have our working dataset of 30 clusters. We'll need to create and drop columns that will make this dataset comparable to *stand_blocks*.

> **Mission: Making each row an "average_block"** - Currently each row in our *demo_blocks_complete* dataset represents a cluster of blocks. As our *stand_blocks* dataset each represent a single block, it will be wise to make the rows in our final dataset simulate an "average block" in each cluster. 

In [13]:
#CLEANING THE cluster COLUMN 
#CURRENTLY THE cluster COLUMN IS A MESSY COLUMN OF...
#...LISTS CONTAINING INTEGERS AND STRINGS NESTED WITHIN A STRING(WHAT?!) 
#WE WANT THEM TO BE A UNIFORM LIST OF STRINGS
#UNSTRINGING EACH LIST WITH THE eval FUNCTION
list_of_cluster_lists = [eval(i) for i in demo_blocks_complete["cluster"]]

In [14]:
#CREATING A LIST OF INTEGERS, BEING CAREFUL NOT TO LEAVE OUT THE BLOCKS THAT ARE 45a or 45b
clean_list_of_cluster_lists = []
for i in list_of_cluster_lists:
    indiv_list=[]
    for item in i:
        if type(item)!=str:
            indiv_list.append(str(item))
        else:
            indiv_list.append(item)
    clean_list_of_cluster_lists.append(indiv_list)
clean_list_of_cluster_lists 


[['6', '7'],
 ['79'],
 ['89'],
 ['454', '455'],
 ['126', '127'],
 ['51', '53', '54'],
 ['580', '581', '582'],
 ['24', '28'],
 ['3', '4', '5'],
 ['20'],
 ['54', '56', '57', '59', '60', '62'],
 ['5', '6', '7', '8', '9', '10'],
 ['180', '181', '182', '184'],
 ['216', '217', '219', '220'],
 ['88', '89', '90', '91', '92'],
 ['246', '247', '248', '249', '250', '251', '252'],
 ['9', '10', '11', '12', '9a', '12a'],
 ['94', '96'],
 ['17',
  '18',
  '19',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31'],
 ['401', '402', '403', '404', '407', '408', '409'],
 ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
 ['74', '75', '76', '77', '78', '79', '80'],
 ['110', '111', '113', '114'],
 ['321', '322', '323'],
 ['1', '2', '3'],
 ['1',
  '2',
  '3',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22'],
 ['167', '168', '169', '170', '171', '172'],
 ['1a', '2a'],
 ['24',
  '25',
  '26',

In [15]:
#CHANGING cluster COLUMN TO FIXED LISTS
demo_blocks_complete["cluster"] = clean_list_of_cluster_lists 

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
demo_blocks_complete

,Site No,SERS Announced,town,street,cluster,year_completed,ave_floors,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,sers_year
7,6,1996-05-28,bedok,chai chee rd,"[6, 7]",1966.0,8.50,0.0,49.0,119.0,0.0,0.0,0.0,168.0,1996
11,10,1997-02-17,toa payoh,toa payoh central,[79],1971.0,10.00,0.0,0.0,374.0,0.0,0.0,0.0,374.0,1997
12,11,1997-02-17,toa payoh,toa payoh lorong 2,[89],1973.0,10.00,0.0,0.0,198.0,0.0,0.0,0.0,198.0,1997
13,12,1997-02-17,ang mo kio,ang mo kio ave 10,"[454, 455]",1979.0,12.00,0.0,0.0,242.0,44.0,0.0,0.0,286.0,1997
14,13,1997-02-17,bukit merah - tiong bahru,kim tian rd,"[126, 127]",1974.0,10.00,0.0,0.0,219.0,0.0,0.0,0.0,219.0,1997
15,14,1997-02-17,bukit merah,nile rd,"[51, 53, 54]",1963.0,10.00,0.0,88.0,270.0,0.0,0.0,0.0,358.0,1997
16,15,1997-04-25,ang mo kio,ang mo kio ave 3,"[580, 581, 582]",1979.0,13.00,0.0,0.0,366.0,70.0,0.0,0.0,436.0,1997
17,16,1997-04-25,queenstown - buona vista,dover cres,"[24, 28]",1975.0,14.00,0.0,0.0,147.0,130.0,0.0,0.0,277.0,1997
18,17,1997-04-25,kallang - kitchener,syed alwi rd,"[3, 4, 5]",1971.0,17.00,0.0,0.0,132.0,43.0,0.0,0.0,175.0,1997
59,57,2005-11-25,kallang - boon keng,upper boon keng rd,[20],1974.0,16.00,0.0,0.0,277.0,0.0,4.0,0.0,281.0,2005


In [17]:
#CREATING A COLUMN THAT HAS THE NUMBER OF BLOCKS IN EACH CLUSTER
demo_blocks_complete["num_of_blocks"]=demo_blocks_complete["cluster"].transform(lambda x: len(x))
demo_blocks_complete["num_of_blocks"]

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


7      2
11     1
12     1
13     2
14     2
15     3
16     3
17     2
18     3
59     1
60     6
64     6
65     4
66     4
67     5
68     7
69     6
70     2
71    13
72     7
73    10
74     7
75     4
76     3
77     3
79    21
80     6
81     2
82    31
83     8
Name: num_of_blocks, dtype: int64

In [18]:
#CREATING A FOR LOOP THAT WILL TRANSFORM xx_room_sold AND total_dwelling_units INTO "AVERAGE UNITS"
#WE DO THAT BY DIVIDING EACH VALUE BY THE num_of_blocks IN EACH CLUSTER
feature_list = ["1room_sold", "2room_sold", "3room_sold", "4room_sold","5room_sold","exec_sold","total_dwelling_units"]
for feature in feature_list:
    demo_blocks_complete[feature]= demo_blocks_complete[feature]/demo_blocks_complete["num_of_blocks"]
demo_blocks_complete

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Site No,SERS Announced,town,street,cluster,year_completed,ave_floors,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,sers_year,num_of_blocks
7,6,1996-05-28,bedok,chai chee rd,"[6, 7]",1966.0,8.50,0.0,24.500000,59.500000,0.000000,0.000000,0.000000,84.000000,1996,2
11,10,1997-02-17,toa payoh,toa payoh central,[79],1971.0,10.00,0.0,0.000000,374.000000,0.000000,0.000000,0.000000,374.000000,1997,1
12,11,1997-02-17,toa payoh,toa payoh lorong 2,[89],1973.0,10.00,0.0,0.000000,198.000000,0.000000,0.000000,0.000000,198.000000,1997,1
13,12,1997-02-17,ang mo kio,ang mo kio ave 10,"[454, 455]",1979.0,12.00,0.0,0.000000,121.000000,22.000000,0.000000,0.000000,143.000000,1997,2
14,13,1997-02-17,bukit merah - tiong bahru,kim tian rd,"[126, 127]",1974.0,10.00,0.0,0.000000,109.500000,0.000000,0.000000,0.000000,109.500000,1997,2
15,14,1997-02-17,bukit merah,nile rd,"[51, 53, 54]",1963.0,10.00,0.0,29.333333,90.000000,0.000000,0.000000,0.000000,119.333333,1997,3
16,15,1997-04-25,ang mo kio,ang mo kio ave 3,"[580, 581, 582]",1979.0,13.00,0.0,0.000000,122.000000,23.333333,0.000000,0.000000,145.333333,1997,3
17,16,1997-04-25,queenstown - buona vista,dover cres,"[24, 28]",1975.0,14.00,0.0,0.000000,73.500000,65.000000,0.000000,0.000000,138.500000,1997,2
18,17,1997-04-25,kallang - kitchener,syed alwi rd,"[3, 4, 5]",1971.0,17.00,0.0,0.000000,44.000000,14.333333,0.000000,0.000000,58.333333,1997,3
59,57,2005-11-25,kallang - boon keng,upper boon keng rd,[20],1974.0,16.00,0.0,0.000000,277.000000,0.000000,4.000000,0.000000,281.000000,2005,1


In [19]:
#ADD AN age COLUMN BY TAKING sers_year MINUS year_completed
#TAKE NOTE THAT THIS WOULD BE THE AGE OF THE BLOCK WHEN THE demolition WAS ANNOUNCED
#INTERESTING TO NOTE THAT WITH TWO ANOMALIES AT 18.0, MOST OF THE FLATS AVERAGE OUT AT ABOUT 30 YEARS 
demo_blocks_complete["age"]= demo_blocks_complete["sers_year"]-demo_blocks_complete["year_completed"]
demo_blocks_complete["age"]

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


7     30.0
11    26.0
12    24.0
13    18.0
14    23.0
15    34.0
16    18.0
17    22.0
18    26.0
59    31.0
60    28.0
64    33.0
65    30.0
66    31.0
67    33.0
68    30.0
69    30.0
70    32.0
71    57.0
72    30.0
73    31.0
74    44.0
75    36.0
76    34.0
77    48.0
79    56.0
80    38.0
81    29.0
82    52.0
83    37.0
Name: age, dtype: float64

In [20]:
#ADDING A BINARY COLUMN TO INDICATE WHEN IT HAS BEEN DEMOLISHED
demo_blocks_complete["is_demolished"]=1
demo_blocks_complete["is_demolished"]

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


7     1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
59    1
60    1
64    1
65    1
66    1
67    1
68    1
69    1
70    1
71    1
72    1
73    1
74    1
75    1
76    1
77    1
79    1
80    1
81    1
82    1
83    1
Name: is_demolished, dtype: int64

---
**Town Codes** - The *stand_blocks* dataset with town codes like AMK for Ang Mo Kio and BH for Bishan, we can match that quite accurately to our *town* column and create a similar column in our dataset

In [21]:
#CHECKING OUT UNIQUE VALUES IN OUR town COLUMN
list(demo_blocks_complete["town"].unique())

['bedok',
 'toa payoh',
 'ang mo kio',
 'bukit merah - tiong bahru',
 'bukit merah',
 'queenstown - buona vista',
 'kallang - kitchener',
 'kallang - boon keng',
 'geylang',
 'jurong west - boon lay gardens',
 'jurong west - taman jurong',
 'bukit merah - silat',
 'clementi',
 'jurong east - teban gardens',
 'queenstown',
 'bedok - east coast',
 'woodlands']

In [22]:
#CREATING A NEW bldg_contract_town COLUMN
demo_blocks_complete["bldg_contract_town"]=demo_blocks_complete["town"].replace({'bedok':"bd",
 'toa payoh': "tp",
 'ang mo kio': "amk",
 'bukit merah - tiong bahru': "bm",
 'bukit merah': "bm",
 'queenstown - buona vista':"qt",
 'kallang - kitchener': "kwn",
 'kallang - boon keng': "kwn",
 'geylang':"gl",
 'jurong west - boon lay gardens': "jw",
 'jurong west - taman jurong': "jw",
 'bukit merah - silat': "bm",
 'clementi': "cl",
 'jurong east - teban gardens': "je",
 'queenstown': "qt",
 'bedok - east coast': "bd",
 'woodlands': "wl"})
demo_blocks_complete["bldg_contract_town"]

/Users/samuelhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


7      bd
11     tp
12     tp
13    amk
14     bm
15     bm
16    amk
17     qt
18    kwn
59    kwn
60     gl
64     jw
65     jw
66     jw
67     bm
68    amk
69     qt
70     bm
71     bm
72     cl
73     je
74     qt
75     bm
76     cl
77     bd
79     bm
80     jw
81     wl
82     qt
83     cl
Name: bldg_contract_town, dtype: object

---
**Exploding Blocks** - We will create a new dataframe that is an "exploded" version of the current one. In the new dataframe, we create one row for every block in a cluster.




In [23]:
#CREATING AN EMPTY DATAFRAME
demo_blocks_indiv = pd.DataFrame(columns=(list(demo_blocks_complete.columns)))
#CREATING A FOR LOOP THAT WILL CREATE ONE ROW IN THE NEW DATAFRAME FOR EVERY BLOCK IN A CLUSTER
for row_num in range(demo_blocks_complete.shape[0]):
    for block in range(demo_blocks_complete.iloc[row_num]["num_of_blocks"]):
        demo_blocks_indiv = demo_blocks_indiv.append(demo_blocks_complete.iloc[row_num])
#RETURNING A NEW DATAFRAME WITH 175 "AVERAGED BLOCKS"
demo_blocks_indiv

,Site No,SERS Announced,town,street,cluster,year_completed,ave_floors,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,sers_year,num_of_blocks,age,is_demolished,bldg_contract_town
7,6,1996-05-28,bedok,chai chee rd,"[6, 7]",1966.0,8.50,0.0,24.5,59.500,0.0,0.0,0.0,84.000,1996,2,30.0,1,bd
7,6,1996-05-28,bedok,chai chee rd,"[6, 7]",1966.0,8.50,0.0,24.5,59.500,0.0,0.0,0.0,84.000,1996,2,30.0,1,bd
11,10,1997-02-17,toa payoh,toa payoh central,[79],1971.0,10.00,0.0,0.0,374.000,0.0,0.0,0.0,374.000,1997,1,26.0,1,tp
12,11,1997-02-17,toa payoh,toa payoh lorong 2,[89],1973.0,10.00,0.0,0.0,198.000,0.0,0.0,0.0,198.000,1997,1,24.0,1,tp
13,12,1997-02-17,ang mo kio,ang mo kio ave 10,"[454, 455]",1979.0,12.00,0.0,0.0,121.000,22.0,0.0,0.0,143.000,1997,2,18.0,1,amk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,79,2016-08-03,clementi,west coast rd,"[513, 514, 515, 516, 517, 518, 519, 520]",1979.0,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,2016,8,37.0,1,cl
83,79,2016-08-03,clementi,west coast rd,"[513, 514, 515, 516, 517, 518, 519, 520]",1979.0,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,2016,8,37.0,1,cl
83,79,2016-08-03,clementi,west coast rd,"[513, 514, 515, 516, 517, 518, 519, 520]",1979.0,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,2016,8,37.0,1,cl
83,79,2016-08-03,clementi,west coast rd,"[513, 514, 515, 516, 517, 518, 519, 520]",1979.0,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,2016,8,37.0,1,cl


In [24]:
#OBTAINING A LIST OF BLOCK NUMBERS FROM THE CLUSTERS
list_of_blk_no=[]
for row_num in range(len(demo_blocks_complete["cluster"])):
    for blk in demo_blocks_complete["cluster"].iloc[row_num]:
        list_of_blk_no.append(blk)
len(list_of_blk_no)

175

In [25]:
#CREATING A COLUMN FOR blk_no
demo_blocks_indiv["blk_no"]= list_of_blk_no
demo_blocks_indiv["blk_no"]

7       6
7       7
11     79
12     89
13    454
     ... 
83    516
83    517
83    518
83    519
83    520
Name: blk_no, Length: 175, dtype: object

---
**Columns** - We'll need to decide which are the final columns we are keeping for our demolished blocks.

In [26]:
#CHECKING COLUMNS IN stand_blocks
stand_blocks.columns

Index(['blk_no', 'street', 'max_floor_lvl', 'year_completed', 'residential',
       'commercial', 'market_hawker', 'miscellaneous', 'multistorey_carpark',
       'precinct_pavilion', 'bldg_contract_town', 'total_dwelling_units',
       '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold',
       'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental',
       '2room_rental', '3room_rental', 'other_room_rental'],
      dtype='object')

In [27]:
#CHECKING COLUMNS IN demo_blocks_complete
demo_blocks_indiv.columns

Index(['Site No', 'SERS Announced', 'town', 'street', 'cluster',
       'year_completed', 'ave_floors', '1room_sold', '2room_sold',
       '3room_sold', '4room_sold', '5room_sold', 'exec_sold',
       'total_dwelling_units', 'sers_year', 'num_of_blocks', 'age',
       'is_demolished', 'bldg_contract_town', 'blk_no'],
      dtype='object')

In [28]:
#RENAMING ave_floors AS IT CORRESPONDS TO max_floor_lvl
#RENAMING age TO age_announced TO BE MORE VERBOSE BECAUSE...
#...age_announced REPRESENTS THE AGE OF THE BLOCK WHEN RESIDENTS ARE INFORMED OF THE DEMOLITION
#age_announced WILL BE USEFUL AT THE EDA PHASE BUT...
#...WE MUST BE VERY CAREFUL IN WHETHER WE WANT TO USE age_announced IN OUR FINAL MODEL
demo_blocks_indiv.rename(columns={'ave_floors':'max_floor_lvl', "age":"age_announced"}, inplace=True)

In [29]:
#CHECKING THE INSECTION OF BOTH DATASETS CURRENTLY
#WE CAN ENGINEER age_announced,  is_demolished,  cluster AND num_of_blocks FROM stand_blocks DATASET...
#...SO WE SHOULD ADD THOSE THREE TO THE FINAL LIST OF COLUMNS

#BUT....WE FIRST NEED TO EXPLODE THIS DATA SET....AND CREATE A blk_no COLUMN
final_columns_list=list(demo_blocks_indiv.columns.intersection(stand_blocks.columns))
final_columns_list.append("age_announced")
final_columns_list.append("cluster")
final_columns_list.append("num_of_blocks")
final_columns_list.append("is_demolished")
final_columns_list

['street',
 'year_completed',
 'max_floor_lvl',
 '1room_sold',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'total_dwelling_units',
 'bldg_contract_town',
 'blk_no',
 'age_announced',
 'cluster',
 'num_of_blocks',
 'is_demolished']

In [30]:
#CREATING A FINAL DATAFRAME WITH COLUMNS THAT WE CAN ACTUALLY GET FROM OUR stand_blocks DATASET
demo_blocks_indiv = demo_blocks_indiv.loc[:, ['year_completed', 'blk_no','street','bldg_contract_town',
                                              'max_floor_lvl', '1room_sold', '2room_sold', 
                                              '3room_sold', '4room_sold', '5room_sold', 
                                              'exec_sold', 'total_dwelling_units',
                                              'age_announced', 'cluster', 'num_of_blocks', 'is_demolished']]
demo_blocks_indiv.reset_index(inplace = True) 
demo_blocks_indiv.drop("index", axis="columns", inplace=True)
demo_blocks_indiv

,year_completed,blk_no,street,bldg_contract_town,max_floor_lvl,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,age_announced,cluster,num_of_blocks,is_demolished
0,1966.0,6,chai chee rd,bd,8.50,0.0,24.5,59.500,0.0,0.0,0.0,84.000,30.0,"[6, 7]",2,1
1,1966.0,7,chai chee rd,bd,8.50,0.0,24.5,59.500,0.0,0.0,0.0,84.000,30.0,"[6, 7]",2,1
2,1971.0,79,toa payoh central,tp,10.00,0.0,0.0,374.000,0.0,0.0,0.0,374.000,26.0,[79],1,1
3,1973.0,89,toa payoh lorong 2,tp,10.00,0.0,0.0,198.000,0.0,0.0,0.0,198.000,24.0,[89],1,1
4,1979.0,454,ang mo kio ave 10,amk,12.00,0.0,0.0,121.000,22.0,0.0,0.0,143.000,18.0,"[454, 455]",2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,1979.0,516,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8,1
171,1979.0,517,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8,1
172,1979.0,518,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8,1
173,1979.0,519,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8,1


In [37]:
demo_blocks_indiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year_completed        175 non-null    float64
 1   blk_no                175 non-null    object 
 2   street                175 non-null    object 
 3   bldg_contract_town    175 non-null    object 
 4   max_floor_lvl         175 non-null    float64
 5   1room_sold            175 non-null    float64
 6   2room_sold            175 non-null    float64
 7   3room_sold            175 non-null    float64
 8   4room_sold            175 non-null    float64
 9   5room_sold            175 non-null    float64
 10  exec_sold             175 non-null    float64
 11  total_dwelling_units  175 non-null    float64
 12  age_announced         175 non-null    float64
 13  cluster               175 non-null    object 
 14  num_of_blocks         175 non-null    object 
 15  is_demolished         1

## 3. Working on the "standing blocks" dataset
Moving on to our standing blocks dataset! We will first engineer the following features: *age_announced,  is_demolished, cluster* and *num_of_blocks*.
We will then choose only columns that exist in *demo_blocks_indiv* before merging both sets of data together!

---
**Creating Clusters** - As blocks rarely go down one by one, its important to pick out clusters. With more than 12000 blocks, it might not be easy picking out clusters visually. Let's start by creating clusters based on blocks with a similar road name built in the same year.  

In [31]:
#CHECKING stand_blocks DATASET
stand_blocks

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,beach rd,16,1970,y,y,n,n,n,n,kwn,142,0,1,138,1,2,0,0,0,0,0,0,0
1,1,bedok sth ave 1,14,1975,y,n,n,y,n,n,bd,206,0,0,204,0,2,0,0,0,0,0,0,0
2,1,cantonment rd,2,2010,n,y,n,n,n,n,ct,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,chai chee rd,15,1982,y,n,n,n,n,n,bd,102,0,0,0,10,92,0,0,0,0,0,0,0
4,1,changi village rd,4,1975,y,y,n,n,n,n,prc,55,0,0,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12195,998a,buangkok cres,18,2018,y,n,n,n,n,n,hg,185,0,102,33,50,0,0,0,0,0,0,0,0
12196,998b,buangkok cres,17,2018,y,n,n,n,n,n,hg,174,0,96,31,47,0,0,0,0,0,0,0,0
12197,999,buangkok cres,2,2018,n,n,n,y,n,y,hg,0,0,0,0,0,0,0,0,0,0,0,0,0
12198,999a,buangkok cres,18,2018,y,n,n,n,n,n,hg,306,0,0,0,0,0,0,0,0,136,170,0,0


In [35]:
#DEFINING A FUNCTION TO HELP CREATE A cluster COLUMN
def make_block_clusters(data):
    #SORTING DATAFRAME 
    data=data.sort_values(["street","year_completed"]).copy()
    #CREATING AN EMPTY LIST
    cluster_list=[]
    cluster_list.clear()
    #DEFINING NEW COLUMNS
    data["cluster"]="dummy_object"
    data["num_of_blocks"]=0
    #DEFINING A ROLLING NUMBER OF BLOCKS
    roll_num_of_blocks=0
    #CREATING A FOR LOOP
    for row in range(data.shape[0]):
        #APPENDING A blk_num INTO A LIST WHENEVER IT QUALIFIES AS PART OF A CLUSTER
        if row<data.shape[0]-1 and data.iloc[row]['street']==data.iloc[row+1]['street'] and data.iloc[row]['year_completed']==data.iloc[row+1]['year_completed']:
            cluster_list.append(str(data.iloc[row]["blk_no"]))
            roll_num_of_blocks+=1
        #ONCE THE CLUSTER IS DONE. WE INSERT IT INTO EVERY BLOCK THAT IS PART OF THE CLUSTER
        else:
            cluster_list.append(str(data.iloc[row]["blk_no"]))
            roll_num_of_blocks+=1
            for i in range(roll_num_of_blocks):
                data.at[data.index[row-i], "cluster"]=cluster_list
                data.at[data.index[row-i], "num_of_blocks"]=len(cluster_list)
            #RETURNING THE BLOCK COUNT TO ZERO
            #RETURNING AN EMPTY CLUSTER LIST
            roll_num_of_blocks=0
            cluster_list=[]
                
    return data

In [36]:
#CALLING THE FUNCTION ON stand_blocks
stand_blocks= make_block_clusters(stand_blocks)
%time
stand_blocks

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,bldg_contract_town,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,cluster,num_of_blocks
5645,356a,admiralty dr,16,1999,y,n,n,y,n,n,sb,85,0,0,0,57,28,0,0,0,0,0,0,0,"[356a, 356b, 356c, 357, 357a, 357b, 357c, 358a...",23
5646,356b,admiralty dr,16,1999,y,n,n,n,n,n,sb,113,0,0,0,32,55,26,0,0,0,0,0,0,"[356a, 356b, 356c, 357, 357a, 357b, 357c, 358a...",23
5647,356c,admiralty dr,16,1999,y,n,n,n,n,n,sb,115,0,0,0,33,56,26,0,0,0,0,0,0,"[356a, 356b, 356c, 357, 357a, 357b, 357c, 358a...",23
5648,357,admiralty dr,16,1999,y,n,n,n,n,n,sb,113,0,0,0,60,29,24,0,0,0,0,0,0,"[356a, 356b, 356c, 357, 357a, 357b, 357c, 358a...",23
5656,357a,admiralty dr,16,1999,y,n,n,n,n,n,sb,113,0,0,0,32,55,26,0,0,0,0,0,0,"[356a, 356b, 356c, 357, 357a, 357b, 357c, 358a...",23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,182,yung sheng rd,18,1999,y,n,n,n,n,n,jw,85,0,0,0,51,34,0,0,0,0,0,0,0,"[177, 178, 179, 180, 181, 182, 183, 184]",8
2286,183,yung sheng rd,18,1999,y,n,n,n,n,n,jw,85,0,0,0,51,34,0,0,0,0,0,0,0,"[177, 178, 179, 180, 181, 182, 183, 184]",8
2310,184,yung sheng rd,18,1999,y,n,n,y,n,n,jw,119,0,0,0,68,51,0,0,0,0,0,0,0,"[177, 178, 179, 180, 181, 182, 183, 184]",8
2273,182a,yung sheng rd,1,2000,n,n,n,n,n,y,jw,0,0,0,0,0,0,0,0,0,0,0,0,0,[182a],1


---
**Creating columns for *is_demolished* and *age_announced*** - We're going to create the remaining columns to match that of the "demolished blocks" dataset. 

***age_announced* caveat** - For the demolished blocks, the values in the *age_announced* column state the age of the block when it was announced that it will be demolished. For the standing blocks, we will input the age of the block in the current year of 2020. Its obviously an odd comparison. This column might be useful during EDA, but we should drop this column before modelling.

In [38]:
#CREATING COLUMNS
stand_blocks["is_demolished"]=0
stand_blocks["age_announced"]=2020 - stand_blocks["year_completed"]

In [39]:
#CHECKING WHETHER stand_blocks HAS EVERY COLUMN IN demo_blocks_indiv
#ANSWER: YES
demo_blocks_indiv.columns.difference(stand_blocks.columns)

Index([], dtype='object')

In [40]:
#THESE ARE THE ADDITIONAL COLUMNS IN stand_blocks
stand_blocks.columns.difference(demo_blocks_indiv.columns)

Index(['1room_rental', '2room_rental', '3room_rental', 'commercial',
       'market_hawker', 'miscellaneous', 'multigen_sold',
       'multistorey_carpark', 'other_room_rental', 'precinct_pavilion',
       'residential', 'studio_apartment_sold'],
      dtype='object')

---
**Standardising both datasets** - We need to make sure the datatypes in each set is the same. let's standardise it. 

In [46]:
#CREATING A DATAFRAME THAT HAS THE SAME COLUMNS
stand_blocks = stand_blocks.loc[:,demo_blocks_indiv.columns.to_list()]

In [49]:
#CHECKING ON DATATYPES
demo_blocks_indiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year_completed        175 non-null    float64
 1   blk_no                175 non-null    object 
 2   street                175 non-null    object 
 3   bldg_contract_town    175 non-null    object 
 4   max_floor_lvl         175 non-null    float64
 5   1room_sold            175 non-null    float64
 6   2room_sold            175 non-null    float64
 7   3room_sold            175 non-null    float64
 8   4room_sold            175 non-null    float64
 9   5room_sold            175 non-null    float64
 10  exec_sold             175 non-null    float64
 11  total_dwelling_units  175 non-null    float64
 12  age_announced         175 non-null    float64
 13  cluster               175 non-null    object 
 14  num_of_blocks         175 non-null    object 
 15  is_demolished         1

In [48]:
#CHECKING ON DATATYPES
stand_blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12200 entries, 5645 to 11615
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year_completed        12200 non-null  int64 
 1   blk_no                12200 non-null  object
 2   street                12200 non-null  object
 3   bldg_contract_town    12200 non-null  object
 4   max_floor_lvl         12200 non-null  int64 
 5   1room_sold            12200 non-null  int64 
 6   2room_sold            12200 non-null  int64 
 7   3room_sold            12200 non-null  int64 
 8   4room_sold            12200 non-null  int64 
 9   5room_sold            12200 non-null  int64 
 10  exec_sold             12200 non-null  int64 
 11  total_dwelling_units  12200 non-null  int64 
 12  age_announced         12200 non-null  int64 
 13  cluster               12200 non-null  object
 14  num_of_blocks         12200 non-null  int64 
 15  is_demolished         12200 non-n

In [52]:
demo_blocks_indiv['num_of_blocks'] = demo_blocks_indiv['num_of_blocks'].astype(float)
demo_blocks_indiv['is_demolished'] = demo_blocks_indiv['is_demolished'].astype(int)

In [54]:
columns_to_float = ['year_completed',
 'max_floor_lvl',
 '1room_sold',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'total_dwelling_units',
 'age_announced',
 'num_of_blocks',]

for col in columns_to_float:
    stand_blocks[col]= stand_blocks[col].astype(float)

In [55]:
#CHECKING ON DATATYPES
stand_blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12200 entries, 5645 to 11615
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year_completed        12200 non-null  float64
 1   blk_no                12200 non-null  object 
 2   street                12200 non-null  object 
 3   bldg_contract_town    12200 non-null  object 
 4   max_floor_lvl         12200 non-null  float64
 5   1room_sold            12200 non-null  float64
 6   2room_sold            12200 non-null  float64
 7   3room_sold            12200 non-null  float64
 8   4room_sold            12200 non-null  float64
 9   5room_sold            12200 non-null  float64
 10  exec_sold             12200 non-null  float64
 11  total_dwelling_units  12200 non-null  float64
 12  age_announced         12200 non-null  float64
 13  cluster               12200 non-null  object 
 14  num_of_blocks         12200 non-null  float64
 15  is_demolished   

In [56]:
#CHECKING ON DATATYPES
#DONE. BOTH OUR DATASETS HAVE THE SAME DATATYPES
demo_blocks_indiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year_completed        175 non-null    float64
 1   blk_no                175 non-null    object 
 2   street                175 non-null    object 
 3   bldg_contract_town    175 non-null    object 
 4   max_floor_lvl         175 non-null    float64
 5   1room_sold            175 non-null    float64
 6   2room_sold            175 non-null    float64
 7   3room_sold            175 non-null    float64
 8   4room_sold            175 non-null    float64
 9   5room_sold            175 non-null    float64
 10  exec_sold             175 non-null    float64
 11  total_dwelling_units  175 non-null    float64
 12  age_announced         175 non-null    float64
 13  cluster               175 non-null    object 
 14  num_of_blocks         175 non-null    float64
 15  is_demolished         1

In [57]:
#GIVING BOTH DATASETS clean NAMES
demo_blocks_clean = demo_blocks_indiv
stand_blocks_clean = stand_blocks

In [99]:
#SAVING THE CREATING DATAFRAME TO A CSV FILE
#COMMENTING OUT THE CODE TO PREVENT DOUBLE SAVES.
#stand_blocks_clean.to_csv('../data/stand_blocks_clean.csv', index = False)
#demo_blocks_clean.to_csv('../data/demo_blocks_clean.csv', index = False)

# PLS DELETE EVERYTHING BELOW THIS LINE

---------

## 4. Scraping for additional features!
We will access the OneMapSG api to get additional features for our dataset.

---
**Exploring the API** - We know that we can obtain location, postal code and long-lat data from the OneMapSG API. But let's first do some quick explorations so see what else we can fish out. 

In [58]:
#INPUT PASSWORD AND USERNAME
Client = OneMapClient("samuelhezhengbang@gmail.com", "Shisha600")


In [98]:
#THIS WILL BE HOW WE PULL OUT LOCATION DATA
Client.search("Ang Mo Kio Street 52")["results"]

[{'SEARCHVAL': 'NIL',
  'BLK_NO': '',
  'ROAD_NAME': 'ANG MO KIO STREET 52',
  'BUILDING': 'NIL',
  'ADDRESS': 'NIL',
  'POSTAL': 'NIL',
  'X': '30254.8071656603',
  'Y': '39386.6491466125',
  'LATITUDE': '1.37247302605048',
  'LONGITUDE': '103.85358064366501',
  'LONGTITUDE': '103.85358064366501'}]

In [93]:
#TRYING MY LUCK WITH
#SOME DEMOLISHED BLOCKS HAVE REINCARNATED AS NEW BLOCKS! THAT'S GOOD!
pd.DataFrame(Client.search("519 WEST COAST ROAD")["results"])

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,WEST COAST PRIDE,519,WEST COAST ROAD,WEST COAST PRIDE,519 WEST COAST ROAD WEST COAST PRIDE SINGAPORE...,120519,19769.6426204955,32220.4911064692,1.30766409641888,103.759364910368,103.759364910368


In [96]:
#THERE'S DATA AVAILABLE ABOUT DIFFERENT PLANNING AREAS IN SINGAPORE
#ONE RELEVANT PIECE OF DATA IS POPULATION BY HOUSING TYPES
#THERE'S DATA AVAILABLE FROM 2000 TO PRESENT
#THIS MIGHT BE WORTH DIGGING INTO FOR SOME EDA. OR JUST ANOTHER PROJECT ALTOGETHER
for i in range(1950, 2020, 1):
    print(Client.get_population_by_dwelling_types(i, 'PASIR RIS'))

{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result': 'No Data Available!'}
{'Result':

In [90]:
demo_blocks_clean

,year_completed,blk_no,street,bldg_contract_town,max_floor_lvl,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,total_dwelling_units,age_announced,cluster,num_of_blocks,is_demolished
0,1966.0,6,chai chee rd,bd,8.50,0.0,24.5,59.500,0.0,0.0,0.0,84.000,30.0,"[6, 7]",2.0,1
1,1966.0,7,chai chee rd,bd,8.50,0.0,24.5,59.500,0.0,0.0,0.0,84.000,30.0,"[6, 7]",2.0,1
2,1971.0,79,toa payoh central,tp,10.00,0.0,0.0,374.000,0.0,0.0,0.0,374.000,26.0,[79],1.0,1
3,1973.0,89,toa payoh lorong 2,tp,10.00,0.0,0.0,198.000,0.0,0.0,0.0,198.000,24.0,[89],1.0,1
4,1979.0,454,ang mo kio ave 10,amk,12.00,0.0,0.0,121.000,22.0,0.0,0.0,143.000,18.0,"[454, 455]",2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,1979.0,516,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8.0,1
171,1979.0,517,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8.0,1
172,1979.0,518,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8.0,1
173,1979.0,519,west coast rd,cl,9.33,0.0,0.0,105.375,17.5,0.5,0.0,123.375,37.0,"[513, 514, 515, 516, 517, 518, 519, 520]",8.0,1


In [61]:
Client.query_api((1,1), {'pageNum': 1})

REFRESHING TOKEN. NEW EXPIRY: 1584583411
'tuple' object has no attribute 'startswith'


In [62]:
??Client.query_api

In [68]:
Client.get_planning_area_names(year=2019)
Client.get_planning_area_names(year=1985)

[{'id': 26, 'pln_area_n': 'SIMPANG'},
 {'id': 27, 'pln_area_n': 'SOUTHERN ISLANDS'},
 {'id': 28, 'pln_area_n': 'SUNGEI KADUT'},
 {'id': 33, 'pln_area_n': 'TUAS'},
 {'id': 34, 'pln_area_n': 'WESTERN ISLANDS'},
 {'id': 35, 'pln_area_n': 'WESTERN WATER CATCHMENT'},
 {'id': 38, 'pln_area_n': 'DOWNTOWN CORE'},
 {'id': 42, 'pln_area_n': 'NEWTON'},
 {'id': 43, 'pln_area_n': 'ORCHARD'},
 {'id': 45, 'pln_area_n': 'KALLANG'},
 {'id': 46, 'pln_area_n': 'LIM CHU KANG'},
 {'id': 49, 'pln_area_n': 'NORTH-EASTERN ISLANDS'},
 {'id': 51, 'pln_area_n': 'PASIR RIS'},
 {'id': 56, 'pln_area_n': 'STRAITS VIEW'},
 {'id': 39, 'pln_area_n': 'MARINA EAST'},
 {'id': 40, 'pln_area_n': 'MARINA SOUTH'},
 {'id': 31, 'pln_area_n': 'TENGAH'},
 {'id': 25, 'pln_area_n': 'SERANGOON'},
 {'id': 57, 'pln_area_n': 'OTHERS'},
 {'id': 5, 'pln_area_n': 'BOON LAY'},
 {'id': 3, 'pln_area_n': 'BEDOK'},
 {'id': 7, 'pln_area_n': 'BUKIT MERAH'},
 {'id': 8, 'pln_area_n': 'BUKIT PANJANG'},
 {'id': 9, 'pln_area_n': 'JURONG EAST'},
 {'id

In [75]:
Client.get_dwelling_types(2020, 'ROCHOR')1.27267256466786	103.822875659974

{'Result': 'No Data Available!'}

In [82]:
Client.get_route(('1.37247302605048', '103.85358064366501'), ('1.27267256466786', '103.822875659974'), "cycle")

{'status_message': 'Found route between points',
 'route_geometry': '}`kG{{zxR?Cl@iDjB^bAL`@Fn@Dx@D`@?dCDGdA?fBJfBZhBDTtBbMBLHf@Hh@PpAHrBAVAr@WzECb@U|Dr@dAx@JpAXjA`@l@XXNRN|A`AVNJHzCjBh@ZnFpBLDLBlCl@h@J`IdB~A`AJJPZBN@`@WrAn@kALM^Ub@KhBA`CT~AHjEBfAO`FGx@EvDh@j@PpH~BtH|BzAZLBLBj@FlG^dCXn@OfAc@h@_@Xc@Ap@Hb@RVx@j@tHd@~DTvCg@JEHGFUCUKSUUc@o@j@`@`A~@Z\\TZPl@Tb@Lf@VB`@Pf@b@p@zArBnE\\Z\\Lt@Pt@Bv@A|Aq@tAo@LBDH?Nt@c@`@MjAQtBOzAI^@`@@lBG`ADxBTzCTp@Dt@?fBa@l@[vBeBn@[dCs@tBo@nBm@pBk@^K`@MdAi@z@aAn@q@f@k@h@a@f@S^ObFuB|B_APGDC~@a@^[f@u@Pk@Dk@Rf@^Xr@Pf@?~@CdA@bBLJ@bAJhBLb@DL@t@FF?\\BtAHj@?J?XAvA_@`@Kd@QJE??dAm@f@_@h@a@zBcBf@In@@d@Nz@p@RXhAxAFH^d@l@\\`@Ln@P`@FND`C`@rAThBZJ@ZFNBl@JvA^`Al@h@l@rDdENUNIPET?VHp@~@FJ~@F`@?~AUdC]zASlEm@nCa@lCs@nBKR@t@HrAZXLXXRTNf@PfCBHBLX~A`@xAZz@FNJNRTl@\\hAb@fA`@lAh@jAf@pDnAj@C~@FtB~@l@V|B`AtAj@|@d@\\d@Nl@@~@nDFv@@`@@h@HbBn@tALlFFAJI~@GpAAP@LBZV~AV\\TP`@Fb@CfBy@tJoE`@QxEuBbGeCfAe@pD}A~@Iv@QhAQVMPUBQAa@U}@OYHGVh@`@~ATXp@XhAPtFl@n@BtAGb@CRCjBi@NEh@QxA_@d@}@H[@UG]VHb@Fj@AHIy@c

In [89]:
Client.get_household_sizes(2015, 'ROCHOR')

[{'planning_area': 'Rochor',
  'person1': 0,
  'person2': 0,
  'person3': 0,
  'person4': 0,
  'person5': 0,
  'person6': None,
  'person7': None,
  'person_more_8': None,
  'year': 2015}]

#TASKS:
#NAME UNIFORMISATION: CLUSTER CHANGE TO BLOCK_NUMBER to match standing blocks, ave_floor change to max_floor_height or whatever

#EXPLODE THE DATA SET INTO MULTIPLE BLOCKS

# NOTES ABOUT METHOD:
- Should my training set be....demolished versus those pre-1980 that were kept. 

- Then the test set is post-1980, these blocks are next! You getit?

# ABOUT LAT + LONG
-Is there a way i can just spread the lat and long out just a tiny bit so that It appears as many dots on the map rather than just one. 

In [ ]:
demo_blocks_indiv['year_completed'].describe()

# VALUE COUNTS BY PERCENTAGE
round(df['quality'].value_counts(normalize = True), 3) * 100

# USING OVERSAMPLING FOR OUR DATASET?

can?
bootstrap??

# USING POLYNOMAIL FEATURES FOR OUR DATASET?

can?

# PCA TUTORIAL HAD A EURISTIC RULE aBOYUT HAVINMG TOO MANY FEATURES


    
- We've clearly overfit our model to the data (so much so that our model's performance is really bad)!
- We have a lot of columns relative to our number of rows! (If you have $n$ rows, it's often advised to keep your number of columns below $\sqrt{n}$.)
</details>

# GRAVE YARD BELOW THIS LINE.
# PLS DELETE

---

## REALLY UGLY CLEANING METHOD. I'VE RECLEANED LIAO

## POOR ATTEMPT TO IMPUTE HEIGHTS OF BLOCKS


In [ ]:
Area_list = Client.get_all_planning_areas(year=2020)

In [ ]:
Names_list = Client.get_planning_area_names(year=2020)

In [ ]:
len(Area_list)

In [ ]:
len(Names_list) 

In [ ]:
Area_list[1]



___

## 1. Data Collection
- For this project, we will be using Reddit's API to collect posts from two subreddits: "r/depression" and "r/SuicideWatch"
- We aim to automate as much of this process as possible into neat functions to enable repeatability on the data collection front.
- When collecting data from servers, we will create a randomized delay between requests as a consideration to Reddit's servers and security staff.

> Note: Data in this notebook was collected on 7 March 2020, 2:30am. Do note that if you run the code again on another day, it will result in a new set of posts being scraped.


### 1.1 Exploring the HTML architecture of the r/depression subreddit page 

In [ ]:
#WE WILL SCRAPE THE r/depression AND r/SuicideWatch SUBREDDITS
#LET'S START BY EXPLORING THE HTML INNARDS OF THE FORMER
url_1 = "https://www.reddit.com/r/depression.json"

In [ ]:
#DEFINING A USER AGENT AND MAKING SURE STATUS IS GOOD TO GO
headers = {"User-agent" : "Sam He"}
res = requests.get(url_1, headers=headers)
res.status_code

In [ ]:
#PEEKING AT WHAT OUR DATA WILL LOOK LIKE
depress_json = res.json()
depress_json

In [ ]:
#THE REDDIT DATA SEEMS TO BE ORGANISED AS A DICTIONARY
#LET'S GET ITS KEYS
sorted(depress_json["data"].keys())

In [ ]:
#WITH SOME HELP FROM THIS YOUTUBE TUTORIAL: https://www.youtube.com/watch?v=5Y3ZE26Ciuk
#WE FIND OUT THAT THE after KEY IS THE QUERY STRING THAT WILL...
#INDICATE IN OUR URL THAT WE WANT TO SEE THE NEXT 25 POSTS AFTER THE after "CODE"

depress_json["data"]["after"]

In [ ]:
#DOUBLE CONFIRMING THAT THE PREVIOUS AFTER KEY IS REALLY THE LAST ITEM ON OUR PAGE
[post["data"]["name"] for post in depress_json["data"]["children"]]

In [ ]:
#CHECKING OUT THE NUMBER OF POSTS IN ONE PAGE
len(depress_json["data"]["children"])

In [ ]:
# OH, WE CAN DATAFRAME IT. 
pd.DataFrame(depress_json["data"]["children"])

In [ ]:
#LOOKS LIKE THIS DATA IS REALLY WHAT WE ARE LOOKING FOR
depress_json["data"]["children"][0]["data"]

### 1.2 Creating functions to automate the Data Collection process 
- We will first run those functions on r/depression and check if they have worked well.

In [ ]:
# NOW WE CAN DEFINE A FUNCTION TO SCRAPE A REDDIT PAGE

def reddit_scrape(url_string, number_of_scrapes, output_list):
    #SCRAPED POSTS WILL BE CONTAINED IN OUTPUT LIST(SHD BE EMPTY)
    #THIS IS USEFUL FOR THE FIRST SCRAPE FROM THE VIRGIN SUBREDDIT
    after = None 
    for _ in range(number_of_scrapes):
        if _ == 0:
            print("SCRAPING {}\n--------------------------------------------------".format(url_string))
            print("<<<SCRAPING COMMENCED>>>") 
            print("Downloading Batch {} of {}...".format(1, number_of_scrapes))
        elif (_+1) % 5 ==0:
            print("Downloading Batch {} of {}...".format((_ + 1), number_of_scrapes))
        
        if after == None:
            params = {}
        else:
            #THIS WILL TELL THE SCRAPER TO GET THE NEXT SET AFTER REDDIT'S after CODE
            params = {"after": after}             
        res = requests.get(url_string, params=params, headers=headers)
        if res.status_code == 200:
            the_json = res.json()
            output_list.extend(the_json["data"]["children"])
            after = the_json["data"]["after"]
        else:
            print(res.status_code)
            break
        time.sleep(randint(1,6))
    
    print("<<<SCRAPING COMPLETED>>>")
    print("Number of posts downloaded: {}".format(len(output_list)))
    print("Number of unique posts: {}".format(len(set([p["data"]["name"] for p in output_list]))))
 

In [ ]:
#CALLING THE FUNCTION ON OUR DEPRESSION SUBREDDIT
depress_scraped = [] #DEFINING AN EMPTY LIST THAT WILL CONTAIN OUR SCRAPED DATA
reddit_scrape("https://www.reddit.com/r/depression.json", 50, depress_scraped)

In [ ]:
#CREATING A FUNCTION TO OUTPUT A LIST OF UNIQUE POSTS
def create_unique_list(original_scrape_list, new_list_name):
    data_name_list=[]
    for i in range(len(original_scrape_list)):
        if original_scrape_list[i]["data"]["name"] not in data_name_list:
            new_list_name.append(original_scrape_list[i]["data"])
            data_name_list.append(original_scrape_list[i]["data"]["name"])
    #CHECKING IF THE NEW LIST IS OF SAME LENGTH AS UNIQUE POSTS
    print("LIST NOW CONTAINS {} UNIQUE SCRAPED POSTS".format(len(new_list_name)))
    

In [ ]:
#CALLING THE FUNCTION ON OUR SCRAPED DATA
depress_scraped_unique = []
create_unique_list(depress_scraped, depress_scraped_unique)

In [ ]:
#PUTTING DEPRESSION DATA INTO A DATAFRAME AND SAVING TO CSV
depression = pd.DataFrame(depress_scraped_unique)
depression["is_suicide"] = 0
depression.head() #CHECK IF THERE ARE 100 COLS AND LAST DUMMY is_suicide COL

### 1.3 Running our functions on the r/SuicideWatch subreddit 

In [ ]:
#CALLING THE SCRAPING FUNCTION ON OUR SUICIDEWATCH SUBREDDIT
suicide_scraped = [] #DEFINING AN EMPTY LIST THAT WILL CONTAIN OUR SCRAPED DATA
reddit_scrape("https://www.reddit.com/r/SuicideWatch.json", 50, suicide_scraped)

In [ ]:
#CALLING THE "UNIQUE ONLY" FUNCTION ON OUR SCRAPED DATA
suicide_scraped_unique = []
create_unique_list(suicide_scraped, suicide_scraped_unique)

In [ ]:
#PUTTING SUICIDEWATCH DATA INTO A DATAFRAME AND SAVING TO CSV
suicide_watch = pd.DataFrame(suicide_scraped_unique)
suicide_watch["is_suicide"] = 1
suicide_watch.head() #CHECK IF THERE ARE 100 COLS AND LAST DUMMY is_suicide COL

#### NOTE: I've commented out the code in the next cell for "pd.to_csv" to prevent any accidental overwriting of the the saved dataset.**

In [ ]:
#suicide_watch.to_csv('../data/suicide_watch.csv', index = False)
#depression.to_csv('../data/depression.csv', index = False)

In [ ]:
#INVESTIGATING THE CASE OF r/SuicideWatch HAVING AN ADDITIONAL COLUMN
suicide_watch.columns.difference(depression.columns)

In [ ]:
#LOOKING INTO THAT ADDITIONAL COLUMN
suicide_watch['author_cakeday'].isnull().value_counts()

#### Early thoughts about the collected data
- Data seems to be collected successfully.
- We have some "uneven-ness" in the size of our set as we collected 980 r/SuicideWatch posts and 917 r/depression posts. We might want to consider "even-ing" out the posts with another round of collection. 
- There is also a matter of r/SuicideWatch having one extra column. Which is strange to me considering that they both exist on the same site. The column is "author_cakeday" and it is mostly NaNs. Thus, it doesn't seem like a column we will be using for our classifier.
